<a href="https://colab.research.google.com/github/Edenshmuel/PapaJohns_Data_Science_Project/blob/main/TFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

In [1]:
# ⬇️ 1. Install & import libraries
!pip install -q pytorch-lightning torchmetrics pytorch-forecasting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 801.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_forecasting import (
    TimeSeriesDataSet, TemporalFusionTransformer, Baseline,
    QuantileLoss)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/cleaned_data.csv')

In [6]:
desc_map = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/desc_encoding_map.csv')
cat_map = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/category_mapping.csv')

In [7]:
df["Date"] = pd.to_datetime(df["Date"])

In [8]:
df = df.rename(columns={"כמות": "quantity"})

In [9]:
# ⬇️ 3. Build a numeric time index (required by PyTorch‑Forecasting)
df = df.sort_values(["clean_desc_encoded", "Date"])
df["time_idx"] = (
    df.groupby("clean_desc_encoded")
      .cumcount())

In [10]:
# ⬇️ 4. Define TFT parameters
max_encoder_length     = 60      # how many historic days the model sees
max_prediction_length  = 30      # horizon (1 week – 1 month, tweak as you like)

In [11]:
training_cutoff = df["time_idx"].max() - max_prediction_length

In [12]:
# ⬇️ 5. Tell the dataset which columns play which role
categorical_static = ["clean_desc_encoded", "category_encoded",
                      "encoded_portion_type"]
categorical_time   = ["Is_Weekend", "Season",
                      "is_christian_holiday", "is_jewish_holiday",
                      "is_near_jewish_holiday", "is_day_before_new_year",
                      "encoded_jewish_holiday", "encoded_christian_holiday"]

In [13]:
# המרה לכל העמודות הקטגוריאליות ל-string ואז ל-category כדי שהמודל יבין שאלו קטגוריות
for col in categorical_static + categorical_time:
    df[col] = df[col].astype(str).astype("category")

In [14]:
# All the real‑valued, time‑varying features
real_time = [
    "Year", "Month", "Day", "WeekOfYear",
    "Day_Name_sin", "Day_Name_cos", "Month_sin", "Month_cos",
    "avg_quantity_all_time", "std_quantity_all_time",
    "num_days_sold", "popularity_score"]

In [15]:
# ⬇️ 6. Wrap everything in a TimeSeriesDataSet
training = TimeSeriesDataSet(
    df[df.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="quantity",
    group_ids=["clean_desc_encoded"],

    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,

    static_categoricals=categorical_static,
    time_varying_known_categoricals=categorical_time,
    time_varying_known_reals=real_time,
    time_varying_unknown_reals=["quantity"],  # the target itself
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,)

validation = TimeSeriesDataSet.from_dataset(
    training, df, min_prediction_idx=training_cutoff+1)

/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries.py:951: UserWarning: Target scales will be only added for continous targets
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries.py:1831: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 67 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__clean_desc_encoded': '103'}, {'__group_id__clean_desc_encoded': '105'}, {'__group_id__clean_desc_encoded': '107'}, {'__group_id__clean_desc_encoded': '111'}, {'__group_id__clean_desc_encoded': '113'}, {'__group_id__clean_desc_encoded': '122'}, {'__group_id__clean_desc_encoded': '123'}, {'__group_id__clean_desc_encoded': '124'}, {'__group_id__clean_desc_encoded': '125'}, {'__group_id__clean_desc_encoded': '126'}]
  warnings.warn(


In [16]:
# ⬇️ 7. Dataloaders
batch_size = 128
train_loader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=2)
val_loader   = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=2)

In [17]:
# ⬇️ 8. Define the model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate      = 1e-3,
    hidden_size        = 32,
    attention_head_size= 4,
    dropout            = 0.1,
    loss               = QuantileLoss(),
    log_interval       = 10,
    reduce_on_plateau_patience = 4,)

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [18]:
import lightning.pytorch as pl

from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import LearningRateMonitor

In [19]:
# ⬇️ 9. Train
trainer = pl.Trainer(
    max_epochs=30,
    accelerator="auto",
    callbacks=[pl.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")])

trainer.fit(tft, train_loader, val_loader)

INFO: You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:lightning.pytorch.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name                               | Type                            | Params | Mode 
-----------------------------------------------------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [64]:
# ⬇️ 10. Forecast next 30 days for all items
preds = tft.predict(val_loader, mode="prediction")   # Tensor on GPU
y_pred = preds.detach().cpu().numpy().flatten()     # numpy‑array שטוח

INFO: You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:lightning.pytorch.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [71]:
# ⬇️ 11. Evaluate
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_true = df.loc[df.time_idx > training_cutoff, "quantity"].values

# ודא שהצורות תואמות
print("y_true shape:", y_true.shape)
print("y_pred shape:", y_pred.shape)
assert y_true.shape == y_pred.shape, "Lengths differ – בדקי time_idx / batch_size"

# 1. MAE & RMSE
mae  = mean_absolute_error(y_true, y_pred)
mse  = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
eps = 1e-8
mape = np.mean(np.abs((y_true - y_pred) / (y_true + eps))) * 100

# 2. SMAPE (Mean Absolute Percentage Error)
eps = 1e-8
smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + eps)) * 100

# 3. MASE (Mean Absolute Scaled Error)
# עקביות: מחשבים את ה‑naive in‑sample error מה‑training
y_train = df.loc[df.time_idx <= training_cutoff, "quantity"].values
naive_errors = np.abs(np.diff(y_train, n=1))
scale = np.mean(naive_errors) if len(naive_errors)>0 else eps
mase = np.mean(np.abs(y_true - y_pred)) / scale

# הדפסה
print(f"MAE:  {mae:.2f} | RMSE: {rmse:.2f}")
print(f"SMAPE: {smape:.2f}% | MASE: {mase:.3f}|MAPE: {mape:.2f}%")

y_true shape: (30,)
y_pred shape: (30,)
MAE:  0.28 | RMSE: 0.68
SMAPE: 15.76% | MASE: 0.877|MAPE: 11.45%


In [75]:
pip install nbformat

In [77]:
import nbformat

# שמו של הקובץ שלך
fname = 'TFT.ipynb'
# קרא בגרסה 4 (הסטנדרטית היום)
nb = nbformat.read(fname, as_version=4)

# הסר metadata.widgets מכל מקום
if 'widgets' in nb.metadata:
    del nb.metadata['widgets']
for cell in nb.cells:
    if 'widgets' in cell.metadata:
        del cell.metadata['widgets']

# כתוב חזרה לקובץ
nbformat.write(nb, fname)
print(f"Cleaned metadata.widgets from {fname}")

FileNotFoundError: [Errno 2] No such file or directory: 'TFT.ipynb'